In [351]:
from typing import Text, Generator, Tuple, List, Optional, Dict, Set
import pandas as pd
import numpy as np
from ast import literal_eval
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import json
import re
import ast
import copy
import imgkit
from tabulate import tabulate
sns.set_theme()

config = imgkit.config(wkhtmltoimage='C:/Program Files/wkhtmltopdf/bin/wkhtmltoimage.exe')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', 5000)
pd.options.display.float_format = '{:.2f}'.format

# 1. Load Datasets

## 1.1 Load Rico GUI JSON Dataset

In [25]:
df_all_guis = pd.read_csv('data/all_guis/all_guis.csv')
df_all_guis['data'] = df_all_guis['data'].apply(ast.literal_eval)

In [26]:
df_all_guis[:1]

,id,data
0,0,"{'bg_color': '#FFFFFF', 'ui_comps': [{'ancestors': ['android.webkit.WebView', 'android.widget.AbsoluteLayout', 'android.view.ViewGroup', 'android.view.View', 'java.lang.Object'], 'bounds': [0, 0, 1440, 2392], 'clickable': True, 'class': 'org.apache.cordova.engine.SystemWebView', 'componentLabel': 'Web View', 'id': '0_0'}], 'ui_comp_groups': []}"


In [27]:
df_all_guis[df_all_guis['id'] == 37166]['data'].values.tolist()

[{'bg_color': '#FAFAFA',
  'ui_comps': [{'text': 'How To',
    'resource-id': 'com.runtastic.android.runtasty.lite:id/bottom_navigation_item_title',
    'ancestors': ['android.widget.TextView',
     'android.view.View',
     'java.lang.Object'],
    'clickable': False,
    'class': 'android.support.v7.widget.AppCompatTextView',
    'bounds': [288, 2320, 576, 2367],
    'componentLabel': 'Text',
    'text_color': '#6F6F6F',
    'bounds_updated': [376.0, 2332.0, 490.66666666666663, 2357.3333333333335],
    'text_updated': 'How To',
    'font_size': 5.91327472470729,
    'id': '37166_0'},
   {'iconClass': 'play',
    'ancestors': ['android.widget.ImageView',
     'android.view.View',
     'java.lang.Object'],
    'resource-id': 'com.runtastic.android.runtasty.lite:id/bottom_navigation_item_icon',
    'clickable': False,
    'class': 'android.support.v7.widget.AppCompatImageView',
    'bounds': [390, 2224, 474, 2308],
    'componentLabel': 'Icon',
    'id': '37166_1'},
   {'text': 'Recipes

## 1.2 Load Test/Train Dataset and Transform

In [318]:
df_user_stories_and_rico = pd.read_csv('data/user_stories/dataset/03_test_train_split/test_us_data_label.csv')
df_user_stories_and_rico['comp_ids'] = df_user_stories_and_rico['comp_ids'].apply(ast.literal_eval)

In [320]:
df_user_stories_and_rico[:5]

,user_story,rico_id,comp_ids,label
0,"As a frequent app user, I want to refresh the content within the home screen, ensuring I have the latest information and resources available to me.",12750,[26],1
1,As a user I want to quickly see how much time a receipe takes for each step to quickyl know how time I woudl need to invest,23369,"[12, 11, 10, 9, 8, 7, 6, 5]",0
2,"As a parent of multiple children, I want to be able to enter the amount of children over or under the age of 12 I am traveling with, so the lower price for them is already included in the prices shown for my results.",16072,"[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]",0
3,"As a PRI user, I want to mark episodes as favorite so that I can watch them again later.",25008,"[11, 5, 0]",1
4,As a user I want to be able to select a league of my choice so I can see the up to date football results,5474,"[60, 61]",1


In [321]:
df_user_stories_and_rico['rico_json_ori'] = df_user_stories_and_rico.apply(lambda row: copy.deepcopy(df_all_guis[df_all_guis['id'] == int(row['rico_id'])]['data'].values.tolist()[0]), axis=1)

In [323]:
df_user_stories_and_rico_train_set = pd.read_csv('data/user_stories/dataset/03_test_train_split/train_us_data_label.csv')
df_user_stories_and_rico_train_set['comp_ids'] = df_user_stories_and_rico_train_set['comp_ids'].apply(ast.literal_eval)
df_user_stories_and_rico_train_set['rico_json_ori'] = df_user_stories_and_rico_train_set.apply(lambda row: copy.deepcopy(df_all_guis[df_all_guis['id'] == int(row['rico_id'])]['data'].values.tolist()[0]), axis=1)

In [326]:
df_user_stories_and_rico_train_set[:4]

user_story  rico_id      comp_ids  label  \
0                                                           As a user i want to be able to read news about transfer of player so I know which player changed to which team    10364  [45, 46, 44]      0   
1  AS a enduser, i want to have a dispaly option at where i can see the total value of my current portfolio so that i have a complete overview over my finacial situation.     6584  [51, 50, 54]      1   
2                                                                                     As a user I want to search for games or clubs to find the clubs of my choice quickly    10364          [48]      1   
3                             As a user I want to add ingredients to a shopping list so that it is easier for me to go shopping and buy everything I need for the cooking.     4891        [4, 2]      1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [39]:
def remove_comps(rico_gui, comp_ids, rico_id):
    # Remove ids from the ui comps
    ori_ui_comps = copy.deepcopy(rico_gui['ui_comps'])
    ids_to_filter = [str(rico_id)+'_'+str(comp_id) for comp_id in comp_ids]
    upd_ui_comps = [elem for elem in ori_ui_comps if not elem['id'] in ids_to_filter]
    rico_gui['ui_comps'] = upd_ui_comps
    # Remove ids from the ui comp groups
    ori_ui_comp_groups = copy.deepcopy(rico_gui['ui_comp_groups'])
    group_ids_to_filter = []
    for group in ori_ui_comp_groups:
        ui_comp_ids_group = group['ui_comp_ids']
        upd_ui_comp_ids_group = [elem for elem in ui_comp_ids_group if elem not in ids_to_filter]
        if not upd_ui_comp_ids_group or len(upd_ui_comp_ids_group) == 0:
            group_ids_to_filter.append(group['id'])
        group['ui_comp_ids'] = upd_ui_comp_ids_group
    ori_ui_comp_groups_updated = [group for group in ori_ui_comp_groups if group['id'] not in group_ids_to_filter]
    rico_gui['ui_comp_groups'] = ori_ui_comp_groups_updated
    return rico_gui

In [ ]:
df_user_stories_and_rico['rico_json_upd'] = df_user_stories_and_rico.apply(lambda row: row['rico_json_ori'] if row['label']==1 else remove_comps(copy.deepcopy(row['rico_json_ori']), row['comp_ids'], row['rico_id']), axis=1)
df_user_stories_and_rico_train_set['rico_json_upd'] = df_user_stories_and_rico_train_set.apply(lambda row: row['rico_json_ori'] if row['label']==1 else remove_comps(copy.deepcopy(row['rico_json_ori']), row['comp_ids'], row['rico_id']), axis=1)

## 1.3 GUI2String

In [549]:
    import ast
    import copy
    import uuid
    from typing import Text, Generator, Tuple, List, Optional, Dict, Set
    import pandas as pd
    import numpy as np
    from ast import literal_eval
    import matplotlib.pyplot as plt
    import seaborn as sns
    import pickle
    import os
    import json
    import re


    FEAT_METHOD_TEXT_ONLY = 'feat_method_text_only'
    FEAT_METHOD_TEXT_COMP_TYPE = 'feat_method_text_comp_type'
    FEAT_METHOD_TEXT_COMP_TYPE_RES_ID = 'feat_method_text_comp_type_res_id'
    FEAT_METHOD_HTML = 'feat_method_html'

    STRUCT_METHOD_SIMPLE_BULLETS = 'struct_method_simple_bullets'
    STRUCT_METHOD_SIMPLE_BULLETS_SORTED = 'struct_method_simple_bullets_sorted'
    STRUCT_METHOD_TWO_LEVEL_BULLETS = 'struct_method_two_level_bullets'
    STRUCT_METHOD_TWO_LEVEL_HTML = 'struct_method_two_level_html'

    STYLE_SIZE = 'style_size'
    STYLE_BACK_COLOR = 'style_back_color'
    STYLE_FONT_COLOR = 'style_font_color'
    STYLE_FONT_SIZE = 'style_font_size'

    stop_words_r_ids = {'main', 'content', 'navigation', 'bar', 'background', 'status',
                        'checkbox', 'widget', 'frame', 'container', 'action', 'btn', 'menu',
                        'label', 'root', 'toolbar', 'view', 'button', 'activity', 'layout',
                        'drawer', 'actionbar', 'icon', 'text', 'banner'}

    html_comp_mapping = {'Web View': ('<div', '</div>'),
                         'Icon': ('<i class="material-icons"', '</i>'),
                         'Button': ('<button type="button"', '</button>'),
                         'Label': ('<p', '</p>'),
                         'Video': ('<video ', '</video> '),
                         'Image': ('<img src="example.jpg"', ''),
                         'Background Image': ('<img src="example.jpg"', ''),
                         'Text': ('<p>', '</p>'),
                         'Checkbox': ('<input type="checkbox"', '</input>'),
                         'Switch': ('<input type="checkbox"', '</input>'),
                         'Text Input': ('<input type="text"', '</input>'),
                         'Input': ('<input type="text"', '</input>'),
                         'Advertisement': ('<div', '</div>'),
                         'Slider': ('<input type="range" min="1" max="100"', '</input>'),
                         'Radio Button': ('<input type="radio"', '</input>'),
                         'Pager Indicator': ('<div', '</div>'),
                         'Map View': ('<div', '</div>')}

    html_comp_group_mapping = {
        'List Item': ('<li', '</li>'),
        'Card': ('<div', '</div>'),
        'Modal': ('<div class="modal"', '</div>'),
        'Map View': ('<div class="map"', '</div>'),
        'Toolbar': ('<menu', '</menu>'),
        'Multi-Tab': ('<div class="tab"', '</div>'),
        'Layout': ('<div class="layout"', '</div>')
    }

    def normalize_resource_id(resource_id: Text, filter_tokens: Optional[Set[Text]] = None,
                              tokenize: Optional[bool] = False) -> List[Text]:
        stopwords = filter_tokens if filter_tokens else stop_words_r_ids
        name_split = resource_id.split('/')
        name = name_split[len(name_split) - 1]
        norm_name = [token for token in snake_camel_case_split(name) if token.lower() not in stopwords]
        return norm_name if tokenize else ' '.join(norm_name)

    def camel_case_split(identifier: Text) -> List[Text]:
        matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
        return [m.group(0) for m in matches]

    def snake_case_split(identifier: Text) -> List[Text]:
        return identifier.split('_')

    def snake_camel_case_split(identifier: Text) -> List[Text]:
        snake_cases = snake_case_split(identifier)
        splits = [cc for sc in snake_cases
                  for cc in camel_case_split(sc)]
        return splits

    def get_refined_comp_type(comp):
        if comp['componentLabel'] == 'On/Off Switch':
            return 'Switch'
        if comp['componentLabel'] == 'Input':
            clazz_name = comp['class'].lower()
            if 'edittext' in clazz_name:
                # return 'Edit Text'
                return 'Text Input'
            elif 'checkbox' in clazz_name:
                return 'Checkbox'
            elif 'switch' in clazz_name:
                return 'Switch'
            else:
                return 'Input'
        if comp['componentLabel'] == 'Text Button':
            clazz_name = comp['class'].lower()
            if 'checkbox' in clazz_name:
                return 'Checkbox'
            else:
                return 'Button'
        if comp['componentLabel'] == 'Text':
            return 'Label'
        return comp['componentLabel']

    def feat_method_text_only(gui, n, m, to_lower, quote, style):
        features = [(ui_comp['id'], ' '.join(ui_comp['text'].split(' ')[:m]))
                    for ui_comp in gui['ui_comps'] if ui_comp.get('text')][:n]
        if to_lower:
            features = [(feat[0], feat[1].lower()) for feat in features]
        if quote:
            features = [(feat[0], '"' + feat[1] + '"') for feat in features]
        return {feat[0]: feat[1] for feat in features}

    def feat_method_text_comp_type(gui, n, m, to_lower, quote, style):
        features = []
        print(gui)
        for ui_comp in gui['ui_comps']:
            uic_text = ui_comp.get('text').strip() if ui_comp.get('text') else ui_comp.get('text_updated').strip() if ui_comp.get('text_updated') else ''
            uic_text = '"' + uic_text + '"' if quote else uic_text
            feat_str = ''
            if ui_comp.get('componentLabel') == 'Icon':
                icon_text = ' '.join(ui_comp.get('iconClass').split('_')).strip()
                icon_text = '"' + icon_text + '"' if quote else icon_text
                feat_str = icon_text + ' (' + get_refined_comp_type(ui_comp) + ')'
            elif ui_comp.get('componentLabel') == 'Text Button':
                if ui_comp.get('buttonClass'):
                    button_text = ' '.join(ui_comp.get('buttonClass').split('_')).strip()
                    button_text = '"' + button_text + '"' if quote else button_text
                    feat_str = button_text + ' (' + get_refined_comp_type(ui_comp) + ')'
                elif ui_comp.get('textButtonClass'):
                    button_text = ' '.join(ui_comp.get('textButtonClass').split('_')).strip()
                    button_text = '"' + button_text + '"' if quote else button_text
                    feat_str = button_text + ' (' + get_refined_comp_type(ui_comp) + ')'
                else:
                    feat_str = uic_text + ' (' + get_refined_comp_type(ui_comp) + ')'
            elif ui_comp.get('componentLabel') == 'Input':
                if ui_comp.get('text'):
                    input_text = '"' + uic_text + '"' if quote else button_text
                    feat_str = input_text + ' (' + get_refined_comp_type(ui_comp) + ')'
                elif ui_comp.get('text_updated'):
                    input_text = '"' + ui_comp.get('text_updated').strip().replace('\n', '').replace('\f', '') + '"' \
                        if quote else ui_comp.get('text_updated').strip().replace('\n', '').replace('\f', '')
                    feat_str = input_text + ' (' + get_refined_comp_type(ui_comp) + ')'
                else:
                    feat_str = '(' + get_refined_comp_type(ui_comp) + ')'
            else:
                if ui_comp.get('text'):
                    feat_str = uic_text + ' (' + get_refined_comp_type(ui_comp) + ')'
                else:
                    feat_str = '(' + get_refined_comp_type(ui_comp) + ')'
            if to_lower:
                feat_str = feat_str.lower()
            features.append((ui_comp.get('id'), feat_str, ui_comp.get('bounds')))
        return {feat[0]: (feat[1], feat[2]) for feat in features}

    def feat_method_text_comp_type_res_id(gui, n, m, to_lower, quote, style, id=False):
        features = []
        for ui_comp in gui['ui_comps']:
            uic_text = ui_comp.get('text').strip() if ui_comp.get('text') else ui_comp.get('text_updated').strip() if ui_comp.get('text_updated') else ''
            uic_text = '"' + uic_text + '"' if quote else uic_text
            feat_str = ''
            if ui_comp.get('componentLabel') == 'Icon':
                icon_text = ' '.join(ui_comp.get('iconClass').split('_')).strip()
                icon_text = '"' + icon_text + '"' if quote else icon_text
                feat_str = icon_text + ' (' + get_refined_comp_type(ui_comp) + ')'
            elif ui_comp.get('componentLabel') == 'Text Button':
                if ui_comp.get('buttonClass'):
                    button_text = ' '.join(ui_comp.get('buttonClass').split('_')).strip()
                    button_text = '"' + button_text + '"' if quote else button_text
                    feat_str = button_text + ' (' + get_refined_comp_type(ui_comp) + ')'
                elif ui_comp.get('textButtonClass'):
                    button_text = ' '.join(ui_comp.get('textButtonClass').split('_')).strip()
                    button_text = '"' + button_text + '"' if quote else button_text
                    feat_str = button_text + ' (' + get_refined_comp_type(ui_comp) + ')'
                else:
                    feat_str = uic_text + ' (' + get_refined_comp_type(ui_comp) + ')'
            elif ui_comp.get('componentLabel') == 'Input':
                if ui_comp.get('text'):
                    input_text = '"' + uic_text + '"' if quote else button_text
                    feat_str = input_text + ' (' + get_refined_comp_type(ui_comp) + ')'
                elif ui_comp.get('text_updated'):
                    input_text = '"' + ui_comp.get('text_updated').strip().replace('\n', '').replace('\f', '') + '"' \
                        if quote else ui_comp.get('text_updated').strip().replace('\n', '').replace('\f', '')
                    feat_str = input_text + ' (' + get_refined_comp_type(ui_comp) + ')'
                else:
                    feat_str = '(' + get_refined_comp_type(ui_comp) + ')'
            else:
                if ui_comp.get('text'):
                    feat_str = uic_text + ' (' + get_refined_comp_type(ui_comp) + ')'
                else:
                    feat_str = '(' + get_refined_comp_type(ui_comp) + ')'
            feat_str += ' (' + normalize_resource_id(ui_comp.get('resource-id')) + ')' if ui_comp.get(
                'resource-id') else ''
            style_attrs = []
            if style.get(STYLE_SIZE):
                bounds = ui_comp['bounds']
                width, height = bounds[2] - bounds[0], bounds[3] - bounds[1]
                style_attrs.append('width:' + str(width))
                style_attrs.append('height:' + str(height))
            if style.get(STYLE_BACK_COLOR) and ui_comp.get('bg_color'):
                style_attrs.append('bg_color:' + ui_comp.get('bg_color'))
            if style.get(STYLE_FONT_COLOR) and ui_comp.get('text_color'):
                style_attrs.append('text_color:' + ui_comp.get('text_color'))
            if style.get(STYLE_FONT_SIZE) and ui_comp.get('font_size'):
                style_attrs.append('font_size:' + str(int(ui_comp.get('font_size'))))
            if style_attrs:
                feat_str = feat_str + ' (' + ';'.join(style_attrs) + ')'
            if to_lower:
                feat_str = feat_str.lower()
            if id:
                feat_str = feat_str + ' (id=' + ui_comp.get('id').split('_')[1] + ')'
            features.append((ui_comp.get('id'), feat_str, ui_comp.get('bounds')))
        return {feat[0]: (feat[1], feat[2]) for feat in features}

    def feat_method_html(gui, n, m, to_lower, quote, style):
        features = []
        for ui_comp in gui['ui_comps']:
            uic_text = ui_comp.get('text').strip() if ui_comp.get('text') else ui_comp.get('text_updated').strip() if ui_comp.get('text_updated') else ''
            html_comp = html_comp_mapping.get(get_refined_comp_type(ui_comp))
            feat_str = html_comp[0]
            if ui_comp.get('resource-id'):
                feat_str += ' id="' + '-'.join(normalize_resource_id(ui_comp.get('resource-id')).split(' ')) + '"'
            style_attrs = []
            if style.get(STYLE_SIZE):
                bounds = ui_comp['bounds']
                width, height = bounds[2] - bounds[0], bounds[3] - bounds[1]
                style_attrs.append('width:' + str(width))
                style_attrs.append('height:' + str(height))
            if style.get(STYLE_BACK_COLOR) and ui_comp.get('bg_color'):
                style_attrs.append('bg_color:' + ui_comp.get('bg_color'))
            if style.get(STYLE_FONT_COLOR) and ui_comp.get('text_color'):
                style_attrs.append('text_color:' + ui_comp.get('text_color'))
            if style.get(STYLE_FONT_SIZE) and ui_comp.get('font_size'):
                style_attrs.append('font_size:' + str(int(ui_comp.get('font_size'))))
            if style_attrs:
                feat_str = feat_str + ' style="' + ';'.join(style_attrs) + '"'
            feat_str += '>'
            if ui_comp.get('componentLabel') == 'Icon':
                icon_text = ' '.join(ui_comp.get('iconClass').split('_')).strip()
                feat_str += icon_text
            elif ui_comp.get('componentLabel') == 'Text Button':
                if ui_comp.get('buttonClass'):
                    button_text = ' '.join(ui_comp.get('buttonClass').split('_')).strip()
                    feat_str += button_text
                elif ui_comp.get('textButtonClass'):
                    button_text = ' '.join(ui_comp.get('textButtonClass').split('_')).strip()
                    feat_str += button_text
                else:
                    feat_str += uic_text
            else:
                if ui_comp.get('text'):
                    feat_str += uic_text
            feat_str += html_comp[1]
            features.append((ui_comp.get('id'), feat_str))
        if to_lower:
            feat_str = feat_str.lower()
        return {feat[0]: (feat[1], feat[2]) for feat in features}

    def features_to_str(gui, feat_method, n, m, to_lower, quote, style, id):
        if feat_method == FEAT_METHOD_TEXT_ONLY:
            return feat_method_text_only(gui, n, m, to_lower, quote, style)
        elif feat_method == FEAT_METHOD_TEXT_COMP_TYPE:
            return feat_method_text_comp_type(gui, n, m, to_lower, quote, style)
        elif feat_method == FEAT_METHOD_TEXT_COMP_TYPE_RES_ID:
            return feat_method_text_comp_type_res_id(gui, n, m, to_lower, quote, style, id)
        elif feat_method == FEAT_METHOD_HTML:
            return feat_method_html(gui, n, m, to_lower, quote, style)

    def filter_uic_groups(uic_groups):
        filtered_uic_groups = []
        # Sort ui comp group based on number of ui comps
        uic_groups_sorted = sorted(uic_groups, key=lambda x: len(x['ui_comp_ids']), reverse=True)
        for i, uic_group_1 in enumerate(uic_groups_sorted, 0):
            subset_count = 0
            for uic_group_2 in uic_groups_sorted[(i + 1):]:
                if uic_group_1['id'] != uic_group_2['id']:
                    if set(uic_group_2['ui_comp_ids']).issubset(uic_group_1['ui_comp_ids']):
                        subset_count += 1
            if subset_count == 0:
                filtered_uic_groups.append(uic_group_1)
        return filtered_uic_groups

    def comp_in_uic(ui_comp_id, ui_comp_groups):
        for uic in ui_comp_groups:
            if ui_comp_id in uic.get('ui_comp_ids'):
                return True
        return False

    def structure_to_str(gui, feat_mappings, struct_method, style):
        gui_cpy = copy.deepcopy(gui)
        gui_mapping = {elem['id']: elem for elem in gui['ui_comps']}
        if struct_method == STRUCT_METHOD_SIMPLE_BULLETS:
            return '\n- ' + '\n- '.join([elem for elem in feat_mappings.values()])
        elif struct_method == STRUCT_METHOD_SIMPLE_BULLETS_SORTED:
            uic_bounds = [(key, val, gui_mapping[key]['bounds']) for key, val in feat_mappings.items()]
            uic_sorted = sorted(uic_bounds, key=lambda x: (x[2][1], x[2][0]))
            return '\n- ' + '\n- '.join([elem[1] for elem in uic_sorted])
        elif struct_method == STRUCT_METHOD_TWO_LEVEL_BULLETS or struct_method == STRUCT_METHOD_TWO_LEVEL_HTML:
            uic_groups = gui_cpy['ui_comp_groups']
            uic_groups = gui_cpy['ui_comp_groups']
            single_ui_comps = [(ui_comp_id, vals[1]) for ui_comp_id, vals in feat_mappings.items()
                               if not comp_in_uic(ui_comp_id, uic_groups)]
            for elem in single_ui_comps:
                uic_groups.append({
                    "componentLabel": "Layout",
                    "bounds": elem[1],
                    "class": "android.widget.LinearLayout",
                    "bg_color": "#FFFFFF",
                    "ui_comp_ids": [elem[0]],
                    'id': str(uuid.uuid4())
                })
            filtered_uic_groups = filter_uic_groups(uic_groups)
            # Find remaining ui comp ids and add them again
            filtered_ui_comp_ids = []
            for fuic in filtered_uic_groups:
                filtered_ui_comp_ids.extend(fuic['ui_comp_ids'])
            filtered_ui_comp_ids = set(filtered_ui_comp_ids)
            all_ui_comp_ids = set([elem['id'] for elem in gui_cpy['ui_comps']])
            missing_ui_comp_ids = all_ui_comp_ids.difference(filtered_ui_comp_ids)
            print('All ids: {}, matched: {}, missing: {}'.format(len(all_ui_comp_ids), len(filtered_ui_comp_ids),
                                                                 len(missing_ui_comp_ids)))
            print('Missing ui comp ids: {}'.format(missing_ui_comp_ids))
            # For the missing ui comp ids, find the smallest ui comp group
            uic_groups_sorted_len = sorted(uic_groups, key=lambda x: len(x['ui_comp_ids']), reverse=False)
            matched_ui_comp_groups = {}
            for miss_ui_comp_id in missing_ui_comp_ids:
                for uic_group_len in uic_groups_sorted_len:
                    if miss_ui_comp_id in uic_group_len['ui_comp_ids']:
                        if uic_group_len['id'] in matched_ui_comp_groups:
                            matched_ui_comp_groups[uic_group_len['id']]['ui_comp_ids'].append(miss_ui_comp_id)
                            break
                        else:
                            matched_ui_comp_groups[uic_group_len['id']] = {
                                "componentLabel": "Layout",
                                "bounds": uic_group_len['bounds'],
                                "class": "android.widget.LinearLayout",
                                "bg_color": "#FFFFFF",
                                "ui_comp_ids": [miss_ui_comp_id],
                                'id': str(uuid.uuid4())
                            }
                            break
            print('additionally matched ui comp groups')
            print(matched_ui_comp_groups)
            print([val for key, val in matched_ui_comp_groups.items()])
            filtered_uic_groups.extend([val for key, val in matched_ui_comp_groups.items()])
            uic_groups_sorted = sorted(filtered_uic_groups, key=lambda x: (x['bounds'][1], x['bounds'][0]))
            feat_str = ''
            for uic_group in uic_groups_sorted:
                uic_group['ui_comp_ids'] = [(feat_mappings.get(idd)[0], gui_mapping.get(idd)) for idd in
                                            uic_group['ui_comp_ids']]
                uic_group['ui_comp_ids'] = sorted(uic_group['ui_comp_ids'],
                                                  key=lambda x: (x[1]['bounds'][1], x[1]['bounds'][0]))
                if struct_method == STRUCT_METHOD_TWO_LEVEL_BULLETS:
                    feat_str += '- ' + uic_group.get('componentLabel')
                    feat_str += '\n\t- ' + '\n\t- '.join([elem[0] for elem in uic_group['ui_comp_ids']]) + '\n'
                elif struct_method == STRUCT_METHOD_TWO_LEVEL_HTML:
                    html_mapping = html_comp_group_mapping.get(uic_group.get('componentLabel'))
                    feat_str += html_mapping[0]
                    style_attrs = []
                    feat_str += '>'
                    feat_str += '\n\t' + '\n\t'.join([elem[0] for elem in uic_group['ui_comp_ids']]) + '\n'
                    feat_str += html_mapping[1] + '\n'
            return feat_str

    def get_str_repr_gui(gui, n, m, to_lower, quote, style, id, feat_method, struct_method):
        feat_mappings = features_to_str(gui, feat_method, n, m, to_lower, quote, style, id)
        final_str = structure_to_str(gui, feat_mappings, struct_method, style)
        return final_str


In [550]:
gui = df_all_guis[df_all_guis['id'] == 12329]['data'].values.tolist()[0]

In [551]:
result = get_str_repr_gui(gui, n=30, m=30, to_lower=False, quote=True, style={}, id=True,
                        feat_method=FEAT_METHOD_TEXT_COMP_TYPE_RES_ID,
                        struct_method=STRUCT_METHOD_TWO_LEVEL_BULLETS)

All ids: 25, matched: 22, missing: 3
Missing ui comp ids: {'12329_16', '12329_17', '12329_18'}
additionally matched ui comp groups
{'12329_28': {'componentLabel': 'Layout', 'bounds': [0, 1387, 1440, 1726], 'class': 'android.widget.LinearLayout', 'bg_color': '#FFFFFF', 'ui_comp_ids': ['12329_16', '12329_17', '12329_18'], 'id': '13a2a655-9c8f-482c-b67e-f41c2c021f77'}}
[{'componentLabel': 'Layout', 'bounds': [0, 1387, 1440, 1726], 'class': 'android.widget.LinearLayout', 'bg_color': '#FFFFFF', 'ui_comp_ids': ['12329_16', '12329_17', '12329_18'], 'id': '13a2a655-9c8f-482c-b67e-f41c2c021f77'}]


In [552]:
print(result)

- Toolbar
	- "arrow backward" (Icon) (id=23)
	- "delete" (Icon) (delete Item) (id=22)
	- "Tuesday Morning Run" (Label) (id=24)
- Layout
	- "December 6, 2016" (Label) (left) (id=20)
	- "10:28 AM" (Label) (right) (id=19)
- Layout
	- "emoji" (Icon) (map Privacy) (id=11)
	- (Image) (id=0)
- Layout
	- "0.00" (Label) (distance) (id=8)
	- "Mi" (Button) (distance Unit) (id=7)
- Layout
	- "00:46" (Label) (time) (id=6)
	- "Duration" (Button) (time Desc) (id=5)
- Layout
	- "0:00" (Label) (avg) (id=4)
	- "Min/mi" (Button) (avg Desc) (id=3)
- Layout
	- "0" (Label) (calories) (id=2)
	- "Calories" (Button) (calories Desc) (id=1)
- Layout
	- "description" (Icon) (notes) (id=18)
	- "Notes" (Label) (notes Title) (id=17)
	- "How did it go, Sam?" (Text Input) (edit Notes) (id=16)
- Layout
	- "photo" (Icon) (image) (id=9)
	- "photo" (Icon) (network Image) (id=10)
- Layout
	- "Share to Facebook" (Label) (left) (id=15)
	- (Switch) (toggle Switch Compat) (id=14)
- Layout
	- "Share to Twitter" (Label) (left) (

In [ ]:
df_user_stories_and_rico['gui_abstraction'] = df_user_stories_and_rico.apply(lambda row: get_str_repr_gui(row['rico_json_upd'], n=30, m=30, to_lower=False, quote=True, style={},
                        feat_method=FEAT_METHOD_TEXT_COMP_TYPE_RES_ID,
                        struct_method=STRUCT_METHOD_TWO_LEVEL_BULLETS), axis=1)

In [962]:
df_user_stories_and_rico.to_csv('data/user_stories/dataset/06_test_train_split_annotated/test_us_data_annotated.csv', index=False)

In [ ]:
df_user_stories_and_rico_train_set['gui_abstraction'] = df_user_stories_and_rico_train_set.apply(lambda row: get_str_repr_gui(row['rico_json_upd'], n=30, m=30, to_lower=False, quote=True, style={},
                        feat_method=FEAT_METHOD_TEXT_COMP_TYPE_RES_ID,
                        struct_method=STRUCT_METHOD_TWO_LEVEL_BULLETS), axis=1)

In [965]:
df_user_stories_and_rico_train_set.to_csv('data/user_stories/dataset/06_test_train_split_annotated/train_us_data_annotated.csv', index=False)

In [988]:
print(df_user_stories_and_rico.iloc[row_id]['gui_abstraction'])

- Toolbar
	- "check" (Icon) (done)
	- "emoji" (Icon) (home)
	- "Preferences" (Label) (title)
	- "arrow backward" (Icon) (up)
- Layout
	- "Preferences" (Label) (title)
- List Item
	- "Show Update Reminder" (Label) (title)
	- (Checkbox) ()
	- "When there is a new version of Calorie Count on Google Play, a friendly reminder will pop up when you open the app." (Label) (summary)
- List Item
	- "Default Home Screen" (Label) (title)
	- "We strongly encourage you to participate in Calorie Camp. But you also have option to change your home screen." (Label) (summary)
- Layout
	- "Foog Log Setting" (Label) (title)
- List Item
	- "Show Complete Day" (Label) (title)
	- "Making a day complete flags it as valid data point for our weekly and monthly analysis tool on the website (available on our website). Incomplete days are ignored." (Label) (summary)
	- (Checkbox) ()
- List Item
	- "Redirect to Calorie Camp" (Label) (title)
	- (Checkbox) ()
	- "Submitting a report to Calorie Camp allows you to earn 

# 2. LLM-GPT Completion API

In [61]:
from openai import OpenAI
organization = ""
api_key = ""
client = OpenAI(api_key=api_key, organization=organization)

In [341]:
def generate_completion(prompt, model='gpt-4', temp=0.75, n=1, max_tokens=15500, logprobs=True, top_logprobs=5, return_obj=True):
    if logprobs:
        chat_completion = client.chat.completions.create(
              model=model,
              messages=[
                    {"role": "user", "content": prompt},
                ],
              temperature=temp,
              n=n,
              logprobs=logprobs,
              top_logprobs=top_logprobs
        )
    else:
        chat_completion = client.chat.completions.create(
              model=model,
              messages=[
                    {"role": "user", "content": prompt},
                ],
              temperature=temp,
              n=n,
        )
    return chat_completion if return_obj else [choice.message.content for choice in chat_completion.choices]

# 3. User Story Implementation Detection

## 3.1 Zero-Shot Prompting Template

In [332]:
PLACEHOLDER_US = 'PLACEHOLDER_US'
PLACEHOLDER_GUI = 'PLACEHOLDER_GUI'
PLACEHOLDER_IMPLEMENTED = '1'
PLACEHOLDER_NOT_IMPLEMENTED = '0'

In [365]:
ZS_TEMPLATE_PRED = "You are given a user story in the context of a graphical user interface. In addition to the user story, you are also given a textual representation of a graphical user interface. The GUI is organized as a two-level bullet point list, the outer points representing layouting groups and the inner points the user interface features contained in that layouting group. Each feature is represented in the following abstract pattern: 'text' (ui component type) (semantic description of ui component), the 'text' being the visual text of the component. Your task is to determine if the user story is already implemented in the given user interface or not. Do not provide any explanation. As your prediction, write '1' if the user story is already implemented otherwise write '0'. \n\nuser story: 'PLACEHOLDER_US' \n\ngraphical user interface description: PLACEHOLDER_GUI \n\nprediction:"

In [366]:
print(ZS_TEMPLATE_PRED)

You are given a user story in the context of a graphical user interface. In addition to the user story, you are also given a textual representation of a graphical user interface. The GUI is organized as a two-level bullet point list, the outer points representing layouting groups and the inner points the user interface features contained in that layouting group. Each feature is represented in the following abstract pattern: 'text' (ui component type) (semantic description of ui component), the 'text' being the visual text of the component. Your task is to determine if the user story is already implemented in the given user interface or not. Do not provide any explanation. As your prediction, write '1' if the user story is already implemented otherwise write '0'. 

user story: 'PLACEHOLDER_US' 

graphical user interface description: PLACEHOLDER_GUI 

prediction:


In [368]:
test_prompt = ZS_TEMPLATE_PRED.replace(PLACEHOLDER_US, df_user_stories_and_rico.iloc[9]['user_story']).replace(PLACEHOLDER_GUI, df_user_stories_and_rico.iloc[row_id]['gui_abstraction'])

In [369]:
print(test_prompt)

You are given a user story in the context of a graphical user interface. In addition to the user story, you are also given a textual representation of a graphical user interface. The GUI is organized as a two-level bullet point list, the outer points representing layouting groups and the inner points the user interface features contained in that layouting group. Each feature is represented in the following abstract pattern: 'text' (ui component type) (semantic description of ui component), the 'text' being the visual text of the component. Your task is to determine if the user story is already implemented in the given user interface or not. Do not provide any explanation. As your prediction, write '1' if the user story is already implemented otherwise write '0'. 

user story: 'As a busy professional, I want to quickly check the store hours of my nearest store so that i can plan my visit according to my schedule' 

graphical user interface description: - Toolbar
	- "search" (Icon) (sear

In [370]:
df_user_stories_and_rico['zero_shot_prompt'] = df_user_stories_and_rico.apply(lambda row: ZS_TEMPLATE_PRED.replace(PLACEHOLDER_US, row['user_story']).replace(PLACEHOLDER_GUI,row['gui_abstraction']), axis=1)
df_user_stories_and_rico['zero_shot_completion'] = df_user_stories_and_rico.apply(lambda row: generate_completion(row['zero_shot_prompt'], model='gpt-4', temp=0), axis=1)
df_user_stories_and_rico['zero_shot_prediction'] = df_user_stories_and_rico.apply(lambda row: 1 if [choice.message.content for choice in row['zero_shot_completion'].choices][0] == PLACEHOLDER_IMPLEMENTED else 0, axis=1)
df_user_stories_and_rico['zero_shot_prediction_prob_0'] = df_user_stories_and_rico.apply(lambda row: [np.exp(top_log_prob.logprob) for top_log_prob in row['zero_shot_completion'].choices[0].logprobs.content[0].top_logprobs if '0' in top_log_prob.token][0], axis=1)
df_user_stories_and_rico['zero_shot_prediction_prob_1'] = df_user_stories_and_rico.apply(lambda row: [np.exp(top_log_prob.logprob) for top_log_prob in row['zero_shot_completion'].choices[0].logprobs.content[0].top_logprobs if '1' in top_log_prob.token][0], axis=1)

## 3.2 Chain-of-Thought Prompting Template

In [423]:
PLACEHOLDER_SPLITTER = 'prediction'

In [424]:
COT_TEMPLATE_PRED = "You are given a user story in the context of a graphical user interface. In addition to the user story, you are also given a textual representation of a graphical user interface. The GUI is organized as a two-level bullet point list, the outer points representing layouting groups and the inner points the user interface features contained in that layouting group. Each feature is represented in the following abstract pattern: 'text' (ui component type) (semantic description of ui component), the 'text' being the visual text of the component. Your task is to determine if the user story is already implemented in the given user interface or not. As your prediction, first write *prediction*: followed by '1' if the user story is already implemented otherwise write '0'.  \n\nuser story: 'PLACEHOLDER_US' \n\ngraphical user interface description: PLACEHOLDER_GUI \n\nLet's think step by step"

In [425]:
print(COT_TEMPLATE_PRED)

You are given a user story in the context of a graphical user interface. In addition to the user story, you are also given a textual representation of a graphical user interface. The GUI is organized as a two-level bullet point list, the outer points representing layouting groups and the inner points the user interface features contained in that layouting group. Each feature is represented in the following abstract pattern: 'text' (ui component type) (semantic description of ui component), the 'text' being the visual text of the component. Your task is to determine if the user story is already implemented in the given user interface or not. As your prediction, first write *prediction*: followed by '1' if the user story is already implemented otherwise write '0'.  

user story: 'PLACEHOLDER_US' 

graphical user interface description: PLACEHOLDER_GUI 

Let's think step by step


In [429]:
df_user_stories_and_rico['cot_prompt'] = df_user_stories_and_rico.apply(lambda row: COT_TEMPLATE_PRED.replace(PLACEHOLDER_US, row['user_story']).replace(PLACEHOLDER_GUI,row['gui_abstraction']), axis=1)
df_user_stories_and_rico['cot_t0_completion'] = df_user_stories_and_rico.apply(lambda row: generate_completion(row['cot_prompt'], model='gpt-4', temp=0), axis=1)
df_user_stories_and_rico['cot_t0_prediction'] = df_user_stories_and_rico.apply(lambda row: 0 if '0' in [choice.message.content for choice in row['cot_t0_completion'].choices][0].lower().split(PLACEHOLDER_SPLITTER)[1].strip() else 1, axis=1)

In [444]:
def get_probs_from_cot_completion(completion, clazz):
    token_list = completion.choices[0].logprobs.content
    final_token = token_list[len(token_list)-1].top_logprobs
    match_prob = [np.exp(top_log_prob.logprob) for top_log_prob in final_token if clazz in top_log_prob.token]
    return match_prob[0] if match_prob else 0

In [445]:
df_user_stories_and_rico['cot_t0_prediction_prob_0'] = df_user_stories_and_rico.apply(lambda row: get_probs_from_cot_completion(row['cot_t0_completion'], '0'), axis=1)
df_user_stories_and_rico['cot_t0_prediction_prob_1'] = df_user_stories_and_rico.apply(lambda row: get_probs_from_cot_completion(row['cot_t0_completion'], '1'), axis=1)
df_user_stories_and_rico['cot_t0_explanation'] = df_user_stories_and_rico.apply(lambda row: [choice.message.content for choice in row['cot_t0_completion'].choices][0].lower().split(PLACEHOLDER_SPLITTER)[0].strip().replace('\n', '').replace('*', ''), axis=1)

In [ ]:
df_user_stories_and_rico['cot_t05_completion'] = df_user_stories_and_rico.apply(lambda row: generate_completion(row['cot_prompt'], model='gpt-4', temp=0.5), axis=1)
df_user_stories_and_rico['cot_t05_prediction'] = df_user_stories_and_rico.apply(lambda row: 0 if '0' in [choice.message.content for choice in row['cot_t05_completion'].choices][0].lower().split(PLACEHOLDER_SPLITTER)[1].strip() else 1, axis=1)
df_user_stories_and_rico['cot_t05_prediction_prob_0'] = df_user_stories_and_rico.apply(lambda row: get_probs_from_cot_completion(row['cot_t05_completion'], '0'), axis=1)
df_user_stories_and_rico['cot_t05_prediction_prob_1'] = df_user_stories_and_rico.apply(lambda row: get_probs_from_cot_completion(row['cot_t05_completion'], '1'), axis=1)
df_user_stories_and_rico['cot_t05_explanation'] = df_user_stories_and_rico.apply(lambda row: [choice.message.content for choice in row['cot_t05_completion'].choices][0].lower().split(PLACEHOLDER_SPLITTER)[0].strip().replace('\n', '').replace('*', ''), axis=1)

In [ ]:
df_user_stories_and_rico['cot_t1_completion'] = df_user_stories_and_rico.apply(lambda row: generate_completion(row['cot_prompt'], model='gpt-4', temp=1), axis=1)
df_user_stories_and_rico['cot_t1_prediction'] = df_user_stories_and_rico.apply(lambda row: 0 if '0' in [choice.message.content for choice in row['cot_t1_completion'].choices][0].lower().split(PLACEHOLDER_SPLITTER)[1].strip() else 1, axis=1)
df_user_stories_and_rico['cot_t1_prediction_prob_0'] = df_user_stories_and_rico.apply(lambda row: get_probs_from_cot_completion(row['cot_t1_completion'], '0'), axis=1)
df_user_stories_and_rico['cot_t1_prediction_prob_1'] = df_user_stories_and_rico.apply(lambda row: get_probs_from_cot_completion(row['cot_t1_completion'], '1'), axis=1)
df_user_stories_and_rico['cot_t1_explanation'] = df_user_stories_and_rico.apply(lambda row: [choice.message.content for choice in row['cot_t1_completion'].choices][0].lower().split(PLACEHOLDER_SPLITTER)[0].strip().replace('\n', '').replace('*', ''), axis=1)

In [ ]:
df_user_stories_and_rico['cot_t13_completion'] = df_user_stories_and_rico.apply(lambda row: generate_completion(row['cot_prompt'], model='gpt-4', max_tokens=1000, temp=1.3), axis=1)

In [497]:
def get_cot_13_prediction(rico_id, completion):
    try:
        return 0 if '0' in [choice.message.content for choice in completion.choices][0].lower().split(PLACEHOLDER_SPLITTER)[1].strip() else 1
    except:
        print(rico_id)
        return 0

In [ ]:
df_user_stories_and_rico['cot_t13_prediction'] = df_user_stories_and_rico.apply(lambda row: get_cot_13_prediction(row['rico_id'], row['cot_t13_completion']), axis=1)
df_user_stories_and_rico['cot_t13_prediction_prob_0'] = df_user_stories_and_rico.apply(lambda row: get_probs_from_cot_completion(row['cot_t13_completion'], '0'), axis=1)
df_user_stories_and_rico['cot_t13_prediction_prob_1'] = df_user_stories_and_rico.apply(lambda row: get_probs_from_cot_completion(row['cot_t13_completion'], '1'), axis=1)
df_user_stories_and_rico['cot_t13_explanation'] = df_user_stories_and_rico.apply(lambda row: [choice.message.content for choice in row['cot_t13_completion'].choices][0].lower().split(PLACEHOLDER_SPLITTER)[0].strip().replace('\n', '').replace('*', ''), axis=1)

## 3.3 Few-Shot Prompting Template

In [383]:
PLACEHOLDER_FS_EXAMPLES = 'PLACEHOLDER_FS_EXAMPLES'
PLACEHOLDER_PREDICTION = 'PLACEHOLDER_PREDICTION'

In [388]:
FS_TEMPLATE_PRED = "You are given a user story in the context of a graphical user interface. In addition to the user story, you are also given a textual representation of a graphical user interface. The GUI is organized as a two-level bullet point list, the outer points representing layouting groups and the inner points the user interface features contained in that layouting group. Each feature is represented in the following abstract pattern: 'text' (ui component type) (semantic description of ui component), the 'text' being the visual text of the component. Your task is to determine if the user story is already implemented in the given user interface or not. Do not provide any explanation. As your prediction, write '1' if the user story is already implemented otherwise write '0'. PLACEHOLDER_FS_EXAMPLES \n\nuser story: 'PLACEHOLDER_US' \n\ngraphical user interface description: PLACEHOLDER_GUI \n\nprediction:"

In [389]:
few_shot_example_templates = []
curr_few_shot_examples = ""
for index, row in df_user_stories_and_rico_train_set.iterrows():
    curr_example = "\n\nuser story: 'PLACEHOLDER_US' \n\ngraphical user interface description: PLACEHOLDER_GUI \n\nprediction:PLACEHOLDER_PREDICTION"
    prediction = '1' if row['label'] == 1 else '0'
    curr_example = curr_example.replace(PLACEHOLDER_US, row['user_story']).replace(PLACEHOLDER_GUI, row['gui_abstraction']).replace(PLACEHOLDER_PREDICTION, prediction)
    curr_few_shot_examples += curr_example
    if (index+1)%5==0:
        fs_template_example = FS_TEMPLATE_PRED.replace(PLACEHOLDER_FS_EXAMPLES, curr_few_shot_examples)
        few_shot_example_templates.append(fs_template_example)
        with open('data/user_stories/dataset/few_shot_prompts/few_shot_prompt_'+str((index+1))+'.txt', "w") as file:
            file.write(fs_template_example)

In [394]:
df_user_stories_and_rico['few_shot_prompt_5'] = df_user_stories_and_rico.apply(lambda row: few_shot_example_templates[0].replace(PLACEHOLDER_US, row['user_story']).replace(PLACEHOLDER_GUI,row['gui_abstraction']), axis=1
df_user_stories_and_rico['few_shot_5_completion'] = df_user_stories_and_rico.apply(lambda row: generate_completion(row['few_shot_prompt_5'], model='gpt-4', temp=0), axis=1)
df_user_stories_and_rico['few_shot_5_prediction'] = df_user_stories_and_rico.apply(lambda row: 1 if [choice.message.content for choice in row['few_shot_5_completion'].choices][0] == PLACEHOLDER_IMPLEMENTED else 0, axis=1)
df_user_stories_and_rico['few_shot_5_prediction_prob_0'] = df_user_stories_and_rico.apply(lambda row: [np.exp(top_log_prob.logprob) for top_log_prob in row['few_shot_5_completion'].choices[0].logprobs.content[0].top_logprobs if '0' in top_log_prob.token][0], axis=1)
df_user_stories_and_rico['few_shot_5_prediction_prob_1'] = df_user_stories_and_rico.apply(lambda row: [np.exp(top_log_prob.logprob) for top_log_prob in row['few_shot_5_completion'].choices[0].logprobs.content[0].top_logprobs if '1' in top_log_prob.token][0], axis=1)

In [411]:
df_user_stories_and_rico['few_shot_prompt_10'] = df_user_stories_and_rico.apply(lambda row: few_shot_example_templates[1].replace(PLACEHOLDER_US, row['user_story']).replace(PLACEHOLDER_GUI,row['gui_abstraction']), axis=1)
df_user_stories_and_rico['few_shot_10_completion'] = df_user_stories_and_rico.apply(lambda row: generate_completion(row['few_shot_prompt_10'], model='gpt-4', temp=0), axis=1)
df_user_stories_and_rico['few_shot_10_prediction'] = df_user_stories_and_rico.apply(lambda row: 1 if [choice.message.content for choice in row['few_shot_10_completion'].choices][0] == PLACEHOLDER_IMPLEMENTED else 0, axis=1)
df_user_stories_and_rico['few_shot_10_prediction_prob_0'] = df_user_stories_and_rico.apply(lambda row: [np.exp(top_log_prob.logprob) for top_log_prob in row['few_shot_10_completion'].choices[0].logprobs.content[0].top_logprobs if '0' in top_log_prob.token][0], axis=1)
df_user_stories_and_rico['few_shot_10_prediction_prob_1'] = df_user_stories_and_rico.apply(lambda row: [np.exp(top_log_prob.logprob) for top_log_prob in row['few_shot_10_completion'].choices[0].logprobs.content[0].top_logprobs if '1' in top_log_prob.token][0], axis=1)

## 3.4 Evaluation

In [31]:
from typing import Text, Callable

class Metric(object):

    def __init__(self, func: Callable, name: Text):
        self.func = func
        self.name = name

In [32]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Precision metric for class 1 (implemented)
precision_1 = lambda y_true, y_pred: precision_score(y_true, y_pred, pos_label=1) 
metric_precision_1 = Metric(func=precision_1, name='Precision_1')
# Precision metric for class 0 (not implemented)
precision_0 = lambda y_true, y_pred: precision_score(y_true, y_pred, pos_label=0) 
metric_precision_0 = Metric(func=precision_0, name='Precision_0')
# Recall metric for class 1 (implemented)
recall_1 = lambda y_true, y_pred: recall_score(y_true, y_pred, pos_label=1) 
metric_recall_1 = Metric(func=recall_1, name='Recall_1')
# Recall metric for class 0 (not implemented)
recall_0 = lambda y_true, y_pred: recall_score(y_true, y_pred, pos_label=0) 
metric_recall_0 = Metric(func=recall_0, name='Recall_0')
# F1-score metric for class 1 (implemented)
f1_1 = lambda y_true, y_pred: f1_score(y_true, y_pred, pos_label=1) 
metric_f1_1 = Metric(func=f1_1, name='F1_1')
# F1-score metric for class 0 (not implemented)
f1_0 = lambda y_true, y_pred: f1_score(y_true, y_pred, pos_label=0) 
metric_f1_0 = Metric(func=f1_0, name='F1_0')
accuracy = lambda y_true, y_pred: accuracy_score(y_true, y_pred) 
metric_accuracy = Metric(func=accuracy, name='Acc.')

In [519]:
def compute_results(y_true, y_pred, metrics, method_name, digits=3, delimiter='|'):
    results = [method_name]
    for metric in metrics:
        metric_val = round(metric.func(y_true, y_pred), digits)
        metric_val = '.' + str(metric_val).split('.')[1]  if metric_val < 1.0 else '1.0'
        results.append(delimiter+str(metric_val))
    return results

In [520]:
result_table = []

In [521]:
# All metrics used for the evaluation of the us prediction problem
metrics = [metric_precision_1, metric_recall_1, metric_f1_1, metric_precision_0, metric_recall_0,  metric_f1_0, metric_accuracy]

In [522]:
# Obtain the true label for the predictions
y_true = df_user_stories_and_rico['label'].values.tolist()

In [524]:
models = [ ('zero_shot_prediction', 'zero_shot'), ('few_shot_5_prediction', 'few_shot_5'), ('few_shot_10_prediction', 'few_shot_10'),
         ('cot_t0_prediction', 'cot_t0'), ('cot_t05_prediction', 'cot_t05'), ('cot_t1_prediction', 'cot_t1'), ('cot_t13_prediction', 'cot_13')]

In [525]:
for model in models:
    y_pred_model = df_user_stories_and_rico[model[0]].values.tolist()
    results_model = compute_results(y_true, y_pred_model, metrics, model[1], delimiter='&')
    result_table.append(results_model) 

In [526]:
# Construct the final table of the overall results
header = ['method']
header.extend([metric.name for metric in metrics])
tabulated_table = tabulate(tabular_data=result_table, headers=header)
print(tabulated_table)

method       Precision_1    Recall_1    F1_1    Precision_0    Recall_0    F1_0    Acc.
-----------  -------------  ----------  ------  -------------  ----------  ------  ------
zero_shot    &.83           &.886       &.857   &.878          &.819       &.847   &.852
few_shot_5   &.829          &.876       &.852   &.869          &.819       &.843   &.848
few_shot_10  &.818          &.857       &.837   &.85           &.81        &.829   &.833
cot_t0       &.9            &.686       &.778   &.746          &.924       &.826   &.805
cot_t05      &.888          &.676       &.768   &.738          &.914       &.817   &.795
cot_t1       &.888          &.752       &.814   &.785          &.905       &.841   &.829
cot_13       &.812          &.743       &.776   &.763          &.829       &.795   &.786


In [9]:
from mlxtend.evaluate import mcnemar_table
from mlxtend.evaluate import mcnemar

def compute_mc_nemar(model_1, model_2, dataset):
    # The correct target (class) labels
    y_target = np.array(df_user_stories_and_rico['label'].values.tolist())
    
    # Class labels predicted by model 1
    y_model1 = np.array(df_user_stories_and_rico[model_1].values.tolist())
    
    # Class labels predicted by model 2
    y_model2 = np.array(df_user_stories_and_rico[model_2].values.tolist())
    
    tb = mcnemar_table(y_target=y_target, 
                       y_model1=y_model1, 
                       y_model2=y_model2)
    chi2, p = mcnemar(ary=tb, corrected=True)
    print('chi-squared:', chi2)
    print('p-value:', p)

In [11]:
compute_mc_nemar('zero_shot_prediction', 'few_shot_5_prediction', df_user_stories_and_rico)

chi-squared: 0.0
p-value: 1.0


In [12]:
compute_mc_nemar('zero_shot_prediction', 'cot_t1_prediction', df_user_stories_and_rico)

chi-squared: 0.5925925925925926
p-value: 0.4414183267820536


In [13]:
compute_mc_nemar('few_shot_5_prediction', 'few_shot_10_prediction', df_user_stories_and_rico)

chi-squared: 0.5714285714285714
p-value: 0.4496917979688908


In [14]:
compute_mc_nemar('few_shot_5_prediction', 'cot_t1_prediction', df_user_stories_and_rico)

chi-squared: 0.32142857142857145
p-value: 0.5707503880581739


In [10]:
compute_mc_nemar('cot_t1_prediction', 'cot_t13_prediction', df_user_stories_and_rico)

chi-squared: 1.8285714285714285
p-value: 0.17629637444050728


In [829]:
df_user_stories_and_rico[['rico_id', 'user_story', 'comp_ids', 'label',
                          'zero_shot_prediction', 'zero_shot_prediction_prob_0', 'zero_shot_prediction_prob_1',
                          'few_shot_5_prediction', 'few_shot_5_prediction_prob_0', 'few_shot_5_prediction_prob_1',
                          'few_shot_10_prediction', 'few_shot_10_prediction_prob_0', 'few_shot_10_prediction_prob_1',
                          'cot_t0_prediction', 'cot_t0_prediction_prob_0', 'cot_t0_prediction_prob_1', 'cot_t0_explanation',
                          'cot_t05_prediction', 'cot_t05_prediction_prob_0', 'cot_t05_prediction_prob_1', 'cot_t05_explanation',
                          'cot_t1_prediction', 'cot_t1_prediction_prob_0', 'cot_t1_prediction_prob_1', 'cot_t1_explanation',
                          'cot_t13_prediction', 'cot_t13_prediction_prob_0', 'cot_t13_prediction_prob_1', 'cot_t13_explanation',
                          'gui_abstraction'
                         ]].to_csv('data/user_stories/dataset/04_test_results_detection/test_us_data_results_detection.csv', index=False)

# 4. User Story Implementation Recommendation

In [ ]:
df_user_stories_and_rico_rec = pd.read_csv('data/user_stories/dataset/05_test_train_split/test_us_data_label.csv')
df_user_stories_and_rico_rec.drop('label', axis=1, inplace=True)
df_user_stories_and_rico_rec['comp_ids'] = df_user_stories_and_rico_rec['comp_ids'].apply(ast.literal_eval)
df_user_stories_and_rico_rec['rico_json_ori'] = df_user_stories_and_rico_rec.apply(lambda row: copy.deepcopy(df_all_guis[df_all_guis['id'] == int(row['rico_id'])]['data'].values.tolist()[0]), axis=1)
df_user_stories_and_rico_rec['rico_json_upd'] = df_user_stories_and_rico_rec.apply(lambda row: remove_comps(copy.deepcopy(row['rico_json_ori']), row['comp_ids'], row['rico_id']), axis=1)
df_user_stories_and_rico_rec['gui_abstraction'] = df_user_stories_and_rico_rec.apply(lambda row: get_str_repr_gui(row['rico_json_upd'], n=30, m=30, to_lower=False, quote=True, style={},
                        feat_method=FEAT_METHOD_TEXT_COMP_TYPE_RES_ID,
                        struct_method=STRUCT_METHOD_TWO_LEVEL_BULLETS), axis=1)

In [76]:
print(df_user_stories_and_rico_rec.iloc[row_index]['gui_abstraction'])

- Toolbar
	- "search" (Icon) (search)
	- "menu" (Icon) (home)
	- "cart" (Icon) (cart imageview)
	- "Store Results" (Label) (title)
- List Item
	- "Stores Near" (Label) (store locator stores near me caption)
	- (Image) (store locator map loader)
	- "20 Stores" (Label) (store locator store results)
	- "Find Stores" (Button) (store locator find stores)
- List Item
	- "1." (Label) (storelocator item position)
	- "SAN FRANCISCO,Store #6498" (Button) (storelocator address line1)
	- "arrow forward" (Icon) (storelocator item navigator)
	- "3700 GEARY BOULEVARD" (Label) (storelocator address line2)
	- "3.24 miles" (Label) (storelocator mile)
	- "SAN FRANCISCO, CA, 94118" (Label) (storelocator citystatezip)
	- "(415) 831-1080" (Button) (storelocator phone contactinfo)
	- (Image) (storelocator brand logo)
	- "Store Hours: 08:00AM-09:00PM" (Label) (storelocator storehours)
	- "Set as My Store" (Button) (storelocator setasmystore)
	- "View Weekly Ads" (Button) (storelocator weeklyads)
- List Item
	

## 4.1 Few-Shot Prompting Template

In [131]:
with open('data/generated/few_shot_prompt_recommendation.txt', 'r') as file:
    few_shot_prompt_recommendation_1 = file.read()

In [132]:
print(few_shot_prompt_recommendation_1)

You are given a user story in the context of a graphical user interface, this user story is not yet implemented. In addition to the user story, you are also given a textual representation of a graphical user interface. The GUI is organized as a two-level bullet point list, the outer points representing layouting groups and the inner points the user interface features contained in that layouting group. Each feature is represented in the following abstract pattern: 'text' (ui component type) (semantic description of ui component), the 'text' being the visual text of the component. Your task implement the feature of the user story in HTML, for styling you can also use CSS. Make sure the generated code can directly be viewed in a browser. Do not provide any explanation. Create HTML code only for the user story, do not repeat the code for the entire GUI. 

user story: 'As a new resident in a city, i want to input my zip code and find the closest office depot so that i can establish where i 

In [832]:
df_user_stories_and_rico_rec['fs_rec_prompt'] = df_user_stories_and_rico_rec.apply(lambda row: few_shot_prompt_recommendation_1.replace(PLACEHOLDER_US, row['user_story']).replace(PLACEHOLDER_GUI, row['gui_abstraction']), axis=1)
df_user_stories_and_rico_rec['fs_rec_gen_result'] = df_user_stories_and_rico_rec.apply(lambda row: generate_completion(row['fs_rec_prompt'], model='gpt-4'), axis=1)
df_user_stories_and_rico_rec['fs_rec_gen_result_parsed'] = df_user_stories_and_rico_rec.apply(lambda row: '<!DOCTYPE html>' + row['fs_rec_gen_result'].choices[0].message.content.replace('\n', '').replace('\t', '').strip(), axis=1)

In [ ]:
def write_html_to_file_and_image(path, model_name dataset):
    for index, row in df_user_stories_and_rico_rec.iterrows():
        file_name = str(index) + '_' + str(row['rico_id'])
        with open(path+file_name+'.html', 'w', encoding='utf-8') as file:
            print(path+file_name+'.html')
            file.write(row[model_name])
        try:
            imgkit.from_string(row[model_name], path+file_name+'.jpg', config=config)
        except:
            pass

In [ ]:
write_html_to_file_and_image('data/generated/html/', 'fs_rec_gen_result_parsed', df_user_stories_and_rico_rec)

## 4.2 (FS-)Chain-of-Thought Prompting Template

In [240]:
with open('data/generated/few_shot_cot_prompt_recommendation.txt', 'r') as file:
    few_shot_cot_prompt_recommendation_1 = file.read()

In [ ]:
HTML_SPLIITER = '*html-code*:'

In [242]:
print(few_shot_cot_prompt_recommendation_1)

You are given a user story in the context of a graphical user interface, this user story is not yet implemented. In addition to the user story, you are also given a textual representation of a graphical user interface. The GUI is organized as a two-level bullet point list, the outer points representing layouting groups and the inner points the user interface features contained in that layouting group. Each feature is represented in the following abstract pattern: 'text' (ui component type) (semantic description of ui component), the 'text' being the visual text of the component. Your task implement the feature of the user story in HTML, for styling you can also use CSS. Make sure the generated code can directly be viewed in a browser. Create HTML code only for the user story, do not repeat the code for the entire GUI.  As for your HTML code generation, first write *html-code*: followed by the actual HTML code.

user story: 'As a new resident in a city, i want to input my zip code and f

In [848]:
df_user_stories_and_rico_rec['fs_cot_rec_prompt'] = df_user_stories_and_rico_rec.apply(lambda row: few_shot_cot_prompt_recommendation_1.replace(PLACEHOLDER_US, row['user_story']).replace(PLACEHOLDER_GUI, row['gui_abstraction']), axis=1)
df_user_stories_and_rico_rec['fs_cot_rec_gen_result'] = df_user_stories_and_rico_rec.apply(lambda row: generate_completion(row['fs_cot_rec_prompt'], model='gpt-4'), axis=1)
df_user_stories_and_rico_rec['fs_cot_rec_gen_result_parsed'] = df_user_stories_and_rico_rec.apply(lambda row: row['fs_cot_rec_gen_result'].choices[0].message.content.split(HTML_SPLIITER)[1].strip(' '), axis=1)

In [ ]:
write_html_to_file_and_image('data/generated/html/', 'fs_cot_rec_gen_result_parsed', df_user_stories_and_rico_rec)

# 5. User Story GUI Component Matching

In [553]:
df_user_stories_and_rico_comp_match = pd.read_csv('data/user_stories/dataset/03_test_train_split/test_us_data_label.csv')

In [ ]:
df_user_stories_and_rico_comp_match['comp_ids'] = df_user_stories_and_rico_comp_match['comp_ids'].apply(ast.literal_eval)
df_user_stories_and_rico_comp_match['rico_json_ori'] = df_user_stories_and_rico_comp_match.apply(lambda row: copy.deepcopy(df_all_guis[df_all_guis['id'] == int(row['rico_id'])]['data'].values.tolist()[0]), axis=1)
df_user_stories_and_rico_comp_match['gui_abstraction'] = df_user_stories_and_rico_comp_match.apply(lambda row: get_str_repr_gui(row['rico_json_ori'], n=30, m=30, to_lower=False, quote=True, style={}, id=True,
                        feat_method=FEAT_METHOD_TEXT_COMP_TYPE_RES_ID,
                        struct_method=STRUCT_METHOD_TWO_LEVEL_BULLETS), axis=1)

In [718]:
df_user_stories_and_rico_comp_match_train = pd.read_csv('data/user_stories/dataset/05_test_train_split/train_us_data_label.csv')

In [ ]:
df_user_stories_and_rico_comp_match_train['comp_ids'] = df_user_stories_and_rico_comp_match_train['comp_ids'].apply(ast.literal_eval)
df_user_stories_and_rico_comp_match_train['rico_json_ori'] = df_user_stories_and_rico_comp_match_train.apply(lambda row: copy.deepcopy(df_all_guis[df_all_guis['id'] == int(row['rico_id'])]['data'].values.tolist()[0]), axis=1)
df_user_stories_and_rico_comp_match_train['gui_abstraction'] = df_user_stories_and_rico_comp_match_train.apply(lambda row: get_str_repr_gui(row['rico_json_ori'], n=30, m=30, to_lower=False, quote=True, style={}, id=True,
                        feat_method=FEAT_METHOD_TEXT_COMP_TYPE_RES_ID,
                        struct_method=STRUCT_METHOD_TWO_LEVEL_BULLETS), axis=1)

## 5.1 Zero-Shot Prompting Template

In [685]:
ZS_TEMPLATE_MATCH_1 = "You are given a user story in the context of a graphical user interface. In addition to the user story, you are also given a textual representation of a graphical user interface. The GUI is organized as a two-level bullet point list, the outer points representing layouting groups and the inner points the user interface features contained in that layouting group. Each feature is represented in the following abstract pattern: 'text' (ui component type) (semantic description of ui component) (id), the 'text' being the visual text of the component and the 'id' being a number identifying each component. Your task is to extract a list of ids belonging to all user interface components that are requried to fulfill the user story. Do not provide any explanation. As your output, write a python list of the ids. \n\nuser story: 'PLACEHOLDER_US' \n\ngraphical user interface description: PLACEHOLDER_GUI \n\noutput:"

In [686]:
print(ZS_TEMPLATE_MATCH_1)

You are given a user story in the context of a graphical user interface. In addition to the user story, you are also given a textual representation of a graphical user interface. The GUI is organized as a two-level bullet point list, the outer points representing layouting groups and the inner points the user interface features contained in that layouting group. Each feature is represented in the following abstract pattern: 'text' (ui component type) (semantic description of ui component) (id), the 'text' being the visual text of the component and the 'id' being a number identifying each component. Your task is to extract a list of ids belonging to all user interface components that are requried to fulfill the user story. Do not provide any explanation. As your output, write a python list of the ids. 

user story: 'PLACEHOLDER_US' 

graphical user interface description: PLACEHOLDER_GUI 

output:


In [705]:
ZS_TEMPLATE_MATCH_2 = "You are given a user story in the context of a graphical user interface. In addition to the user story, you are also given a textual representation of a graphical user interface. The GUI is organized as a two-level bullet point list, the outer points representing layouting groups and the inner points the user interface features contained in that layouting group. Each feature is represented in the following abstract pattern: 'text' (ui component type) (semantic description of ui component) (id), the 'text' being the visual text of the component and the 'id' being a number identifying each component. Your task is to extract a list of ids belonging to all user interface components that are requried to fulfill the user story. Make sure to extract all components, e.g. also labels that provide information to the user and are part of a feature. Do not provide any explanation. As your output, write a python list of the ids. \n\nuser story: 'PLACEHOLDER_US' \n\ngraphical user interface description: PLACEHOLDER_GUI \n\noutput:"

In [706]:
print(ZS_TEMPLATE_MATCH_2)

You are given a user story in the context of a graphical user interface. In addition to the user story, you are also given a textual representation of a graphical user interface. The GUI is organized as a two-level bullet point list, the outer points representing layouting groups and the inner points the user interface features contained in that layouting group. Each feature is represented in the following abstract pattern: 'text' (ui component type) (semantic description of ui component) (id), the 'text' being the visual text of the component and the 'id' being a number identifying each component. Your task is to extract a list of ids belonging to all user interface components that are requried to fulfill the user story. Make sure to extract all components, e.g. also labels that provide information to the user and are part of a feature. Do not provide any explanation. As your output, write a python list of the ids. 

user story: 'PLACEHOLDER_US' 

graphical user interface description:

In [689]:
df_user_stories_and_rico_comp_match['zero_shot_prompt_match_1'] = df_user_stories_and_rico_comp_match.apply(lambda row: ZS_TEMPLATE_MATCH_1.replace(PLACEHOLDER_US, row['user_story']).replace(PLACEHOLDER_GUI,row['gui_abstraction']), axis=1)

In [690]:
print(df_user_stories_and_rico_comp_match.iloc[3]['zero_shot_prompt_match_1'])

You are given a user story in the context of a graphical user interface. In addition to the user story, you are also given a textual representation of a graphical user interface. The GUI is organized as a two-level bullet point list, the outer points representing layouting groups and the inner points the user interface features contained in that layouting group. Each feature is represented in the following abstract pattern: 'text' (ui component type) (semantic description of ui component) (id), the 'text' being the visual text of the component and the 'id' being a number identifying each component. Your task is to extract a list of ids belonging to all user interface components that are requried to fulfill the user story. Do not provide any explanation. As your output, write a python list of the ids. 

user story: 'As a PRI user, I want to mark episodes as favorite so that I can watch them again later.' 

graphical user interface description: - Toolbar
	- "menu" (Icon) (id=28)
	- (Imag

In [691]:
def compute_binary_y(y_true, y_pred):
    y_true_set = set(y_true)
    y_pred_set = set(y_pred)
    intersection = y_true_set.intersection(y_pred_set)
    y_true = []
    y_pred = []
    for i in range(len(intersection)):
        y_true.append(1)
        y_pred.append(1)
    diff_pred = y_pred_set.difference(y_true_set)
    for i in range(len(diff_pred)):
        y_true.append(0)
        y_pred.append(1)
    diff_true = y_true_set.difference(y_pred_set)
    for i in range(len(diff_true)):
        y_true.append(1)
        y_pred.append(0)
    return (y_true, y_pred)

In [692]:
def compute_match_eval(y_true, y_pred):
    y_true_set = set(y_true)
    y_pred_set = set(y_pred)
    intersection = y_true_set.intersection(y_pred_set)
    precision = len(intersection) / len(y_pred_set) if len(y_pred_set) > 0 else 0
    recall = len(intersection) / len(y_true_set) if len(y_true_set) > 0 else 0
    f1 = 2* (precision*recall) / (precision+recall) if (precision+recall) != 0 else 0
    return (precision, recall, f1)

In [ ]:
df_user_stories_and_rico_comp_match['zero_shot_match_1_prediction'] = df_user_stories_and_rico_comp_match.apply(lambda row: generate_completion(row['zero_shot_prompt_match_1'], model='gpt-4', temp=0, n=1, return_obj=False)[0], axis=1)
df_user_stories_and_rico_comp_match['zero_shot_match_1_prediction_parsed'] = df_user_stories_and_rico_comp_match['zero_shot_match_1_prediction'].apply(ast.literal_eval)
df_user_stories_and_rico_comp_match['zero_shot_match_1_prediction_parsed'] = df_user_stories_and_rico_comp_match['zero_shot_match_1_prediction_parsed'].apply(lambda x: [str(elem) for elem in x])
df_user_stories_and_rico_comp_match['zero_shot_match_1_eval_mac'] = df_user_stories_and_rico_comp_match.apply(lambda row: compute_match_eval(row['comp_ids'], row['zero_shot_match_1_prediction_parsed']), axis=1)
df_user_stories_and_rico_comp_match['zero_shot_match_1_eval_mic'] = df_user_stories_and_rico_comp_match.apply(lambda row: compute_binary_y(row['comp_ids'], row['zero_shot_match_1_prediction_parsed']), axis=1)

In [701]:
df_user_stories_and_rico_comp_match[['comp_ids', 'zero_shot_match_1_prediction_parsed', 'zero_shot_match_1_eval_mac', 'zero_shot_match_1_eval_mic']][:10]

,comp_ids,zero_shot_match_1_prediction_parsed,zero_shot_match_1_eval_mac,zero_shot_match_1_eval_mic
0,[26],[26],"(1.0, 1.0, 1.0)","([1], [1])"
1,"[12, 11, 10, 9, 8, 7, 6, 5]","[12, 11, 10, 9, 8, 7]","(1.0, 0.75, 0.8571428571428571)","([1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0])"
2,"[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]","[8, 12, 10, 3, 2, 0]","(0.6666666666666666, 0.4, 0.5)","([1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0])"
3,"[11, 5, 0]","[11, 5, 0, 21]","(0.75, 1.0, 0.8571428571428571)","([1, 1, 1, 0], [1, 1, 1, 1])"
4,"[60, 61]","[60, 55, 53, 51, 49, 47, 45, 43, 41]","(0.1111111111111111, 0.5, 0.1818181818181818)","([1, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 0])"
5,"[1, 2]","[15, 14, 1]","(0.3333333333333333, 0.5, 0.4)","([1, 0, 0, 1], [1, 1, 1, 0])"
6,[5],[],"(0, 0.0, 0)","([1], [0])"
7,"[10, 9, 8, 7, 6]","[10, 9, 8, 7, 6]","(1.0, 1.0, 1.0)","([1, 1, 1, 1, 1], [1, 1, 1, 1, 1])"
8,[3],[3],"(1.0, 1.0, 1.0)","([1], [1])"
9,"[16, 5]","[23, 22, 16]","(0.3333333333333333, 0.5, 0.4)","([1, 0, 0, 1], [1, 1, 1, 0])"


In [707]:
df_user_stories_and_rico_comp_match['zero_shot_prompt_match_2'] = df_user_stories_and_rico_comp_match.apply(lambda row: ZS_TEMPLATE_MATCH_2.replace(PLACEHOLDER_US, row['user_story']).replace(PLACEHOLDER_GUI,row['gui_abstraction']), axis=1)
df_user_stories_and_rico_comp_match['zero_shot_match_2_prediction'] = df_user_stories_and_rico_comp_match.apply(lambda row: generate_completion(row['zero_shot_prompt_match_2'], model='gpt-4', temp=0, n=1, return_obj=False)[0], axis=1)
df_user_stories_and_rico_comp_match['zero_shot_match_2_prediction_parsed'] = df_user_stories_and_rico_comp_match['zero_shot_match_2_prediction'].apply(ast.literal_eval)
df_user_stories_and_rico_comp_match['zero_shot_match_2_prediction_parsed'] = df_user_stories_and_rico_comp_match['zero_shot_match_2_prediction_parsed'].apply(lambda x: [str(elem) for elem in x])
df_user_stories_and_rico_comp_match['zero_shot_match_2_eval_mac'] = df_user_stories_and_rico_comp_match.apply(lambda row: compute_match_eval(row['comp_ids'], row['zero_shot_match_2_prediction_parsed']), axis=1)
df_user_stories_and_rico_comp_match['zero_shot_match_2_eval_mic'] = df_user_stories_and_rico_comp_match.apply(lambda row: compute_binary_y(row['comp_ids'], row['zero_shot_match_2_prediction_parsed']), axis=1)

In [713]:
df_user_stories_and_rico_comp_match[['comp_ids', 'zero_shot_match_2_prediction_parsed', 'zero_shot_match_2_eval_mac', 'zero_shot_match_2_eval_mic']][:10]

,comp_ids,zero_shot_match_2_prediction_parsed,zero_shot_match_2_eval_mac,zero_shot_match_2_eval_mic
0,[26],[26],"(1.0, 1.0, 1.0)","([1], [1])"
1,"[12, 11, 10, 9, 8, 7, 6, 5]","[12, 11, 10, 9, 8, 7]","(1.0, 0.75, 0.8571428571428571)","([1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0])"
2,"[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]","[8, 12, 10, 11, 3, 7, 5, 6]","(0.625, 0.5, 0.5555555555555556)","([1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0])"
3,"[11, 5, 0]","[11, 5]","(1.0, 0.6666666666666666, 0.8)","([1, 1, 1], [1, 1, 0])"
4,"[60, 61]","[60, 55, 53, 51, 49, 47, 45, 43, 41]","(0.1111111111111111, 0.5, 0.1818181818181818)","([1, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 0])"
5,"[1, 2]","[15, 14, 1, 2]","(0.5, 1.0, 0.6666666666666666)","([1, 1, 0, 0], [1, 1, 1, 1])"
6,[5],[],"(0, 0.0, 0)","([1], [0])"
7,"[10, 9, 8, 7, 6]","[10, 9, 8, 7, 6]","(1.0, 1.0, 1.0)","([1, 1, 1, 1, 1], [1, 1, 1, 1, 1])"
8,[3],"[2, 3]","(0.5, 1.0, 0.6666666666666666)","([1, 0], [1, 1])"
9,"[16, 5]","[25, 26, 27, 24, 23, 22, 21, 20, 19, 12, 13, 11, 16, 10, 9, 8, 1, 2, 0, 5]","(0.1, 1.0, 0.18181818181818182)","([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])"


## 5.2 Chain-of-Thought Prompting Template

In [ ]:
PLACEHOLDER_SPLITTER = 'prediction'
COT_TEMPLATE_PRED = "You are given a user story in the context of a graphical user interface. In addition to the user story, you are also given a textual representation of a graphical user interface. The GUI is organized as a two-level bullet point list, the outer points representing layouting groups and the inner points the user interface features contained in that layouting group. Each feature is represented in the following abstract pattern: 'text' (ui component type) (semantic description of ui component), the 'text' being the visual text of the component. Your task is to determine if the user story is already implemented in the given user interface or not. As your prediction, first write *prediction*: followed by '1' if the user story is already implemented otherwise write '0'.  \n\nuser story: 'PLACEHOLDER_US' \n\ngraphical user interface description: PLACEHOLDER_GUI \n\nLet's think step by step"
df_user_stories_and_rico['cot_t0_prediction'] = df_user_stories_and_rico.apply(lambda row: 0 if '0' in [choice.message.content for choice in row['cot_t0_completion'].choices][0].lower().split(PLACEHOLDER_SPLITTER)[1].strip() else 1, axis=1)

In [769]:
PLACEHOLDER_SPLITTER_MATCH = '*output*:'
PLACEHOLDER_SPLITTER_MATCH_ALT = 'output:'
PLACEHOLDER_SPLITTER_SIMPLE = 'output'

In [740]:
COT_TEMPLATE_MATCH = "You are given a user story in the context of a graphical user interface. In addition to the user story, you are also given a textual representation of a graphical user interface. The GUI is organized as a two-level bullet point list, the outer points representing layouting groups and the inner points the user interface features contained in that layouting group. Each feature is represented in the following abstract pattern: 'text' (ui component type) (semantic description of ui component) (id), the 'text' being the visual text of the component and the 'id' being a number identifying each component. Your task is to extract a list of ids belonging to all user interface components that are requried to fulfill the user story As your output, first write *output*: followed by a python list of the ids. \n\nuser story: 'PLACEHOLDER_US' \n\ngraphical user interface description: PLACEHOLDER_GUI \n\nLet's think step by step"

In [741]:
print(COT_TEMPLATE_MATCH)

You are given a user story in the context of a graphical user interface. In addition to the user story, you are also given a textual representation of a graphical user interface. The GUI is organized as a two-level bullet point list, the outer points representing layouting groups and the inner points the user interface features contained in that layouting group. Each feature is represented in the following abstract pattern: 'text' (ui component type) (semantic description of ui component) (id), the 'text' being the visual text of the component and the 'id' being a number identifying each component. Your task is to extract a list of ids belonging to all user interface components that are requried to fulfill the user story As your output, first write *output*: followed by a python list of the ids. 

user story: 'PLACEHOLDER_US' 

graphical user interface description: PLACEHOLDER_GUI 

Let's think step by step


In [809]:
def extract_id_list(prediction):
    splits = prediction.lower().split(PLACEHOLDER_SPLITTER_MATCH)
    if len(splits)==2:
        return ast.literal_eval(splits[1].replace('*', '').replace('\\', '').replace('"','').strip())
    else:
        alt_splits = prediction.lower().split(PLACEHOLDER_SPLITTER_MATCH_ALT)
        return ast.literal_eval(alt_splits[1].replace('*', '').replace('\\', '').replace('"','').strip()) if len(alt_splits) == 2 else []

def extract_id_list_simplified(prediction):
    start = prediction.find('[')
    end = prediction.find(']', start)
    list_str = prediction[start:end+1]
    try:
        return ast.literal_eval(list_str)
    except:
        print('exception')
        return []

In [746]:
df_user_stories_and_rico_comp_match['cot_prompt_match'] = df_user_stories_and_rico_comp_match.apply(lambda row: COT_TEMPLATE_MATCH.replace(PLACEHOLDER_US, row['user_story']).replace(PLACEHOLDER_GUI,row['gui_abstraction']), axis=1)
df_user_stories_and_rico_comp_match['cot_prompt_match_t0_prediction'] = df_user_stories_and_rico_comp_match.apply(lambda row: generate_completion(row['cot_prompt_match'], model='gpt-4', temp=0, n=1, max_tokens=2000, return_obj=False)[0], axis=1)
df_user_stories_and_rico_comp_match['cot_match_t0_prediction_parsed'] = df_user_stories_and_rico_comp_match.apply(lambda row: extract_id_list(row['cot_prompt_match_t0_prediction']), axis=1)
df_user_stories_and_rico_comp_match['cot_match_t0_prediction_parsed'] = df_user_stories_and_rico_comp_match['cot_match_t0_prediction_parsed'].apply(lambda x: [str(elem) for elem in x])
df_user_stories_and_rico_comp_match['cot_match_t0_prediction_explanation'] = df_user_stories_and_rico_comp_match.apply(lambda row: row['cot_prompt_match_t0_prediction'].split(PLACEHOLDER_SPLITTER_SIMPLE)[0].replace('\n', '').replace('*', ''), axis=1)
df_user_stories_and_rico_comp_match['cot_match_t0_eval_mac'] = df_user_stories_and_rico_comp_match.apply(lambda row: compute_match_eval(row['comp_ids'], row['cot_match_t0_prediction_parsed']), axis=1)
df_user_stories_and_rico_comp_match['cot_match_t0_eval_mic'] = df_user_stories_and_rico_comp_match.apply(lambda row: compute_binary_y(row['comp_ids'], row['cot_match_t0_prediction_parsed']), axis=1)

In [753]:
df_user_stories_and_rico_comp_match.iloc[1]['cot_prompt_match_t0_prediction']

'The user story states that the user wants to quickly see how much time a recipe takes for each step. This means the user interface components that display the time for each step of the recipe are required. From the graphical user interface description, these components are:\n\n- "Preparation :" (Label) (id=12)\n- "20 min" (Label) (preparation Time) (id=11)\n- "Cooking :" (Label) (id=10)\n- "3 min" (Label) (cooking Time) (id=9)\n- "Total :" (Label) (total Time Header) (id=8)\n- "23 min" (Label) (total Time) (id=7)\n\nSo, the ids of these components are 12, 11, 10, 9, 8, and 7.\n\n*output*: [12, 11, 10, 9, 8, 7]'

In [ ]:
df_user_stories_and_rico_comp_match['cot_prompt_match_t1_prediction'] = df_user_stories_and_rico_comp_match.apply(lambda row: generate_completion(row['cot_prompt_match'], model='gpt-4', temp=1, n=1, max_tokens=2000, return_obj=False)[0], axis=1)
df_user_stories_and_rico_comp_match['cot_match_t1_prediction_parsed'] = df_user_stories_and_rico_comp_match.apply(lambda row: extract_id_list(row['cot_prompt_match_t1_prediction']), axis=1)
df_user_stories_and_rico_comp_match['cot_match_t1_prediction_parsed'] = df_user_stories_and_rico_comp_match['cot_match_t1_prediction_parsed'].apply(lambda x: [str(elem) for elem in x])
df_user_stories_and_rico_comp_match['cot_match_t1_prediction_explanation'] = df_user_stories_and_rico_comp_match.apply(lambda row: row['cot_prompt_match_t1_prediction'].split(PLACEHOLDER_SPLITTER_SIMPLE)[0].replace('\n', '').replace('*', ''), axis=1)
df_user_stories_and_rico_comp_match['cot_match_t1_eval_mac'] = df_user_stories_and_rico_comp_match.apply(lambda row: compute_match_eval(row['comp_ids'], row['cot_match_t1_prediction_parsed']), axis=1)
df_user_stories_and_rico_comp_match['cot_match_t1_eval_mic'] = df_user_stories_and_rico_comp_match.apply(lambda row: compute_binary_y(row['comp_ids'], row['cot_match_t1_prediction_parsed']), axis=1)

In [ ]:
df_user_stories_and_rico_comp_match['cot_prompt_match_t05_prediction'] = df_user_stories_and_rico_comp_match.apply(lambda row: generate_completion(row['cot_prompt_match'], model='gpt-4', temp=0.5, n=1, max_tokens=2000, return_obj=False)[0], axis=1)
df_user_stories_and_rico_comp_match['cot_match_t05_prediction_parsed'] = df_user_stories_and_rico_comp_match.apply(lambda row: extract_id_list(row['cot_prompt_match_t05_prediction']), axis=1)
df_user_stories_and_rico_comp_match['cot_match_t05_prediction_parsed'] = df_user_stories_and_rico_comp_match['cot_match_t05_prediction_parsed'].apply(lambda x: [str(elem) for elem in x])
df_user_stories_and_rico_comp_match['cot_match_t05_prediction_explanation'] = df_user_stories_and_rico_comp_match.apply(lambda row: row['cot_prompt_match_t05_prediction'].split(PLACEHOLDER_SPLITTER_SIMPLE)[0].replace('\n', '').replace('*', ''), axis=1)
df_user_stories_and_rico_comp_match['cot_match_t05_eval_mac'] = df_user_stories_and_rico_comp_match.apply(lambda row: compute_match_eval(row['comp_ids'], row['cot_match_t05_prediction_parsed']), axis=1)
df_user_stories_and_rico_comp_match['cot_match_t05_eval_mic'] = df_user_stories_and_rico_comp_match.apply(lambda row: compute_binary_y(row['comp_ids'], row['cot_match_t05_prediction_parsed']), axis=1)

In [ ]:
df_user_stories_and_rico_comp_match['cot_prompt_match_t13_prediction'] = df_user_stories_and_rico_comp_match.apply(lambda row: generate_completion(row['cot_prompt_match'], model='gpt-4', temp=1.3, n=1, max_tokens=2000, return_obj=False)[0], axis=1)
df_user_stories_and_rico_comp_match['cot_match_t13_prediction_parsed'] = df_user_stories_and_rico_comp_match.apply(lambda row: extract_id_list_simplified(row['cot_prompt_match_t13_prediction']), axis=1)
df_user_stories_and_rico_comp_match['cot_match_t13_prediction_parsed'] = df_user_stories_and_rico_comp_match['cot_match_t13_prediction_parsed'].apply(lambda x: [str(elem) for elem in x])
df_user_stories_and_rico_comp_match['cot_match_t13_prediction_explanation'] = df_user_stories_and_rico_comp_match.apply(lambda row: row['cot_prompt_match_t13_prediction'].split(PLACEHOLDER_SPLITTER_SIMPLE)[0].replace('\n', '').replace('*', ''), axis=1)
df_user_stories_and_rico_comp_match['cot_match_t13_eval_mac'] = df_user_stories_and_rico_comp_match.apply(lambda row: compute_match_eval(row['comp_ids'], row['cot_match_t13_prediction_parsed']), axis=1)
df_user_stories_and_rico_comp_match['cot_match_t13_eval_mic'] = df_user_stories_and_rico_comp_match.apply(lambda row: compute_binary_y(row['comp_ids'], row['cot_match_t13_prediction_parsed']), axis=1)

## 5.3 Few-Shot Prompting Template

In [715]:
PLACEHOLDER_FS_EXAMPLES = 'PLACEHOLDER_FS_EXAMPLES'
PLACEHOLDER_PREDICTION = 'PLACEHOLDER_PREDICTION'

In [716]:
FS_TEMPLATE_MATCH = "You are given a user story in the context of a graphical user interface. In addition to the user story, you are also given a textual representation of a graphical user interface. The GUI is organized as a two-level bullet point list, the outer points representing layouting groups and the inner points the user interface features contained in that layouting group. Each feature is represented in the following abstract pattern: 'text' (ui component type) (semantic description of ui component) (id), the 'text' being the visual text of the component and the 'id' being a number identifying each component. Your task is to extract a list of ids belonging to all user interface components that are requried to fulfill the user story. Do not provide any explanation. As your output, write a python list of the ids. PLACEHOLDER_FS_EXAMPLES \n\nuser story: 'PLACEHOLDER_US' \n\ngraphical user interface description: PLACEHOLDER_GUI \n\noutput:"

In [717]:
print(FS_TEMPLATE_MATCH)

You are given a user story in the context of a graphical user interface. In addition to the user story, you are also given a textual representation of a graphical user interface. The GUI is organized as a two-level bullet point list, the outer points representing layouting groups and the inner points the user interface features contained in that layouting group. Each feature is represented in the following abstract pattern: 'text' (ui component type) (semantic description of ui component) (id), the 'text' being the visual text of the component and the 'id' being a number identifying each component. Your task is to extract a list of ids belonging to all user interface components that are requried to fulfill the user story. Do not provide any explanation. As your output, write a python list of the ids. PLACEHOLDER_FS_EXAMPLES 

user story: 'PLACEHOLDER_US' 

graphical user interface description: PLACEHOLDER_GUI 

output:


In [726]:
few_shot_example_templates = []
curr_few_shot_examples = ""
for index, row in df_user_stories_and_rico_comp_match_train.iterrows():
    curr_example = "\n\nuser story: 'PLACEHOLDER_US' \n\ngraphical user interface description: PLACEHOLDER_GUI \n\noutput:PLACEHOLDER_PREDICTION"
    prediction = str([int(elem) for elem in row['comp_ids']])
    curr_example = curr_example.replace(PLACEHOLDER_US, row['user_story']).replace(PLACEHOLDER_GUI, row['gui_abstraction']).replace(PLACEHOLDER_PREDICTION, prediction)
    curr_few_shot_examples += curr_example
    if (index+1)%5==0:
        fs_template_example = FS_TEMPLATE_MATCH.replace(PLACEHOLDER_FS_EXAMPLES, curr_few_shot_examples)
        few_shot_example_templates.append(fs_template_example)
        with open('data/user_stories/dataset/few_shot_prompts/extraction/few_shot_prompt_'+str((index+1))+'.txt', "w") as file:
            file.write(fs_template_example)
df_user_stories_and_rico['few_shot_prompt_5'] = df_user_stories_and_rico.apply(lambda row: few_shot_example_templates[0].replace(PLACEHOLDER_US, row['user_story']).replace(PLACEHOLDER_GUI,row['gui_abstraction']), axis=1)

In [727]:
df_user_stories_and_rico_comp_match['few_shot_prompt_match_5'] = df_user_stories_and_rico_comp_match.apply(lambda row: few_shot_example_templates[0].replace(PLACEHOLDER_US, row['user_story']).replace(PLACEHOLDER_GUI,row['gui_abstraction']), axis=1)
df_user_stories_and_rico_comp_match['few_shot_prompt_match_5_prediction'] = df_user_stories_and_rico_comp_match.apply(lambda row: generate_completion(row['few_shot_prompt_match_5'], model='gpt-4', temp=0, n=1, return_obj=False)[0], axis=1)
df_user_stories_and_rico_comp_match['few_shot_prompt_match_5_prediction_parsed'] = df_user_stories_and_rico_comp_match['few_shot_prompt_match_5_prediction'].apply(ast.literal_eval)
df_user_stories_and_rico_comp_match['few_shot_prompt_match_5_prediction_parsed'] = df_user_stories_and_rico_comp_match['few_shot_prompt_match_5_prediction_parsed'].apply(lambda x: [str(elem) for elem in x])
df_user_stories_and_rico_comp_match['few_shot_prompt_match_5_eval_mac'] = df_user_stories_and_rico_comp_match.apply(lambda row: compute_match_eval(row['comp_ids'], row['few_shot_prompt_match_5_prediction_parsed']), axis=1)
df_user_stories_and_rico_comp_match['few_shot_prompt_match_5_eval_mic'] = df_user_stories_and_rico_comp_match.apply(lambda row: compute_binary_y(row['comp_ids'], row['few_shot_prompt_match_5_prediction_parsed']), axis=1)

In [732]:
df_user_stories_and_rico_comp_match[['comp_ids', 'few_shot_prompt_match_5_prediction_parsed', 'few_shot_prompt_match_5_eval_mac', 'few_shot_prompt_match_5_eval_mic']][:10]

,comp_ids,few_shot_prompt_match_5_prediction_parsed,few_shot_prompt_match_5_eval_mac,few_shot_prompt_match_5_eval_mic
0,[26],[26],"(1.0, 1.0, 1.0)","([1], [1])"
1,"[12, 11, 10, 9, 8, 7, 6, 5]","[11, 9, 7]","(1.0, 0.375, 0.5454545454545454)","([1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 0, 0, 0, 0, 0])"
2,"[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]","[12, 10, 11, 7, 5, 6, 2, 0, 1]","(0.6666666666666666, 0.6, 0.631578947368421)","([1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])"
3,"[11, 5, 0]","[11, 5, 0]","(1.0, 1.0, 1.0)","([1, 1, 1], [1, 1, 1])"
4,"[60, 61]",[60],"(1.0, 0.5, 0.6666666666666666)","([1, 1], [1, 0])"
5,"[1, 2]","[1, 2]","(1.0, 1.0, 1.0)","([1, 1], [1, 1])"
6,[5],[],"(0, 0.0, 0)","([1], [0])"
7,"[10, 9, 8, 7, 6]","[10, 9, 8, 7, 6]","(1.0, 1.0, 1.0)","([1, 1, 1, 1, 1], [1, 1, 1, 1, 1])"
8,[3],[3],"(1.0, 1.0, 1.0)","([1], [1])"
9,"[16, 5]","[16, 20]","(0.5, 0.5, 0.5)","([1, 0, 1], [1, 1, 0])"


## 5.4 Evaluation

In [37]:
def beautify_score(score, digits):
    score = round(score, digits)
    score = '.' + str(score).split('.')[1]  if score < 1.0 else '1.0'
    return score

def compute_results(dataset, method, digits=3, delimiter='|'):
    results = [method[2]]
    # Compute macro values
    eval_results = dataset[method[0]].values.tolist()
    macro_precision = np.mean([elem[0] for elem in eval_results])
    macro_recall = np.mean([elem[1] for elem in eval_results])
    macro_f1 = np.mean([elem[2] for elem in eval_results])
    results.append(delimiter+str(beautify_score(macro_precision, digits=digits)))
    results.append(delimiter+str(beautify_score(macro_recall, digits=digits)))
    results.append(delimiter+str(beautify_score(macro_f1, digits=digits)))
    # Compute micro values
    eval_results_mic = dataset[method[1]].values.tolist()
    y_true = [elem[0] for elem in eval_results_mic]
    y_true = [item for elem in y_true for item in elem]
    y_pred = [elem[1] for elem in eval_results_mic]
    y_pred = [item for elem in y_pred for item in elem]
    results.append(delimiter+str(beautify_score(precision_score(y_true, y_pred), digits=digits)))
    results.append(delimiter+str(beautify_score(recall_score(y_true, y_pred), digits=digits)))
    results.append(delimiter+str(beautify_score(f1_score(y_true, y_pred), digits=digits)))
    results.append(delimiter+str(beautify_score(accuracy_score(y_true, y_pred), digits=digits)))
    return results

result_table = []

models = [('zero_shot_match_1_eval_mac', 'zero_shot_match_1_eval_mic', 'zero_shot_1'), 
          ('zero_shot_match_2_eval_mac', 'zero_shot_match_2_eval_mic', 'zero_shot_2'),
         ('few_shot_prompt_match_5_eval_mac', 'few_shot_prompt_match_5_eval_mic', 'few_shot_5'),
         ('cot_match_t0_eval_mac', 'cot_match_t0_eval_mic', 'cot_0'),
          ('cot_match_t05_eval_mac', 'cot_match_t05_eval_mic', 'cot_05'),
         ('cot_match_t1_eval_mac', 'cot_match_t1_eval_mic', 'cot_1'),
         ('cot_match_t13_eval_mac', 'cot_match_t13_eval_mic', 'cot_13')]

for model in models:
    results_model = compute_results(df_user_stories_and_rico_comp_match, model, delimiter='&')
    result_table.append(results_model)

# Construct the final table of the overall results
header = ['method']
header.extend(['Mac-Precision', 'Mac-Recall', 'Mac-F1', 'Mic-Precision', 'Mic-Recall', 'Mic-F1', 'Mic-Acc'])
tabulated_table = tabulate(tabular_data=result_table, headers=header)
print(tabulated_table)

method       Mac-Precision    Mac-Recall    Mac-F1    Mic-Precision    Mic-Recall    Mic-F1    Mic-Acc
-----------  ---------------  ------------  --------  ---------------  ------------  --------  ---------
zero_shot_1  &.784            &.819         &.755     &.62             &.755         &.681     &.516
zero_shot_2  &.718            &.903         &.743     &.502            &.858         &.633     &.463
few_shot_5   &.85             &.755         &.765     &.677            &.643         &.659     &.492
cot_0        &.758            &.806         &.727     &.557            &.731         &.632     &.462
cot_05       &.721            &.788         &.688     &.492            &.71          &.581     &.409
cot_1        &.698            &.809         &.69      &.534            &.746         &.622     &.452
cot_13       &.677            &.74          &.654     &.562            &.646         &.601     &.43


In [40]:
from scipy.stats import wilcoxon

def compute_wilcoxon_signed_rank_test(model_1, model_2, metric, dataset):
    eval_results_mac_1 = dataset[model_1].values.tolist()
    y_prec_model_1 = [elem[metric] for elem in eval_results_mac_1]
    eval_results_mac_2 = dataset[model_2].values.tolist()
    y_prec_model_2 = [elem[metric] for elem in eval_results_mac_2]
    print(wilcoxon(y_prec_model_1, y_prec_model_2))

In [43]:
compute_wilcoxon_signed_rank_test('zero_shot_match_1_eval_mac', 'zero_shot_match_2_eval_mac', 2, df_user_stories_and_rico_comp_match)

WilcoxonResult(statistic=1774.5, pvalue=0.5545757800578714)


In [45]:
compute_wilcoxon_signed_rank_test('zero_shot_match_1_eval_mac', 'few_shot_prompt_match_5_eval_mac', 2, df_user_stories_and_rico_comp_match)

WilcoxonResult(statistic=2483.5, pvalue=0.8863503269755024)


In [48]:
compute_wilcoxon_signed_rank_test('zero_shot_match_1_eval_mac', 'cot_match_t0_eval_mac', 2, df_user_stories_and_rico_comp_match)

WilcoxonResult(statistic=1267.0, pvalue=0.12575821715768823)


In [49]:
compute_wilcoxon_signed_rank_test('zero_shot_match_1_eval_mac', 'cot_match_t1_eval_mac', 2, df_user_stories_and_rico_comp_match)

WilcoxonResult(statistic=1440.0, pvalue=0.0007472199498632703)


In [50]:
compute_wilcoxon_signed_rank_test('few_shot_prompt_match_5_eval_mac', 'cot_match_t1_eval_mac', 2, df_user_stories_and_rico_comp_match)

WilcoxonResult(statistic=2709.0, pvalue=0.00243841048489151)


In [51]:
compute_wilcoxon_signed_rank_test('few_shot_prompt_match_5_eval_mac', 'cot_match_t0_eval_mac', 2, df_user_stories_and_rico_comp_match)

WilcoxonResult(statistic=2420.5, pvalue=0.04269059783284989)


In [823]:
df_user_stories_and_rico_comp_match[['rico_id', 'user_story', 'comp_ids', 'zero_shot_match_1_prediction_parsed', 'zero_shot_match_1_eval_mac',
                                     'zero_shot_match_2_prediction_parsed', 'zero_shot_match_2_eval_mac', 'few_shot_prompt_match_5_prediction_parsed',
                                     'few_shot_prompt_match_5_eval_mac', 'cot_match_t0_prediction_parsed', 'cot_match_t0_eval_mac', 'cot_match_t0_prediction_explanation',
                                     'cot_match_t05_prediction_parsed', 'cot_match_t05_eval_mac', 'cot_match_t05_prediction_explanation',
                                     'cot_match_t1_prediction_parsed', 'cot_match_t1_eval_mac', 'cot_match_t1_prediction_explanation',
                                     'cot_match_t13_prediction_parsed', 'cot_match_t13_eval_mac', 'cot_match_t13_prediction_explanation',
                                     'gui_abstraction']].to_csv('data/user_stories/dataset/05_test_results_matching/test_us_data_results_matching.csv', index=False)